In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from itertools import combinations,product




Есть продажи некоторых продуктов по дням отдельно в каждый канал продаж (Shipments_by_PO.csv) на период с января 17 по июнь 18. Также указано, с какой скидкой был продан продукт. Продажи могут быть не во все дни.
  * Есть привязка каждого канала продаж к т.н. направлению продаж – оптовая или розничная торговля (Mapping.csv)
  * Есть прогноз, с какой скидкой будут продаваться продукты в июле-августе 18 (Forecast_of_discounts.xlsx). Если продукта/направления нет в файле, значит скидка – 0%
  * Нужно построить прогноз продаж на июль-сентябрь 18 с помощью простейшего алгоритма – линейной регрессии.
  * Регрессию нужно построить по каждой комбинации продукт-направление отдельно, учитывая номер месяца, года и размер скидки. Номер месяца нужно преобразовать с помощью One-Hot Encoding.
  * Прогноз нужен на уровне месяца, продукта и направления продаж, соответственно, все данные нужно сначала преобразовать на данный уровень с помощью Pandas.

In [4]:
data_shipments = pd.read_csv('C:/Users/Peter/Documents/GitHub/PeterOstr/test_run/Shipments_by_PO.csv')
data_mapping = pd.read_csv('C:/Users/Peter/Documents/GitHub/PeterOstr/test_run/Mapping.csv')
data_forecast = pd.read_excel('C:/Users/Peter/Documents/GitHub/PeterOstr/test_run/Forecast_of_discounts.xlsx')


In [5]:
data_forecast

,Код товара,Направление продаж,Month,Year,Скидка
0,SKU_0001,Розничная торговля,7,2018,10.000000
1,SKU_0002,Оптовая торговля,7,2018,7.400000
2,SKU_0004,Розничная торговля,8,2018,15.500000
3,SKU_0005,Оптовая торговля,9,2018,10.857143
4,SKU_0006,Оптовая торговля,7,2018,12.500000
...,...,...,...,...,...
1100,SKU_0654,Розничная торговля,7,2018,15.000000
1101,SKU_0654,Розничная торговля,9,2018,4.500000
1102,SKU_0656,Розничная торговля,8,2018,12.000000
1103,SKU_0657,Оптовая торговля,7,2018,14.428571


In [6]:
a = data_forecast['Код товара'].value_counts()
len(a)

573

In [7]:
#проверим на наличие пропусков
{key:data_forecast[key].isna().sum() for key in data_forecast.columns}


{'Код товара': 0, 'Направление продаж': 0, 'Month': 0, 'Year': 0, 'Скидка': 0}

In [8]:
#проверим на наличие пропусков

{key:data_shipments[key].isna().sum() for key in data_shipments.columns}


{'Код товара': 0,
 'Канал': 0,
 'Дата накладной': 0,
 'Продажи, шт': 0,
 'скидка': 0}

In [9]:
#добавил столбец направление продаж

data = data_shipments.copy()
data = pd.merge(data, data_mapping, on='Канал', how='left')
data

,Код товара,Канал,Дата накладной,"Продажи, шт",скидка,Направление продаж
0,299,Канал 1,06.01.2017,32,19,Розничная торговля
1,299,Канал 1,09.01.2017,59,19,Розничная торговля
2,299,Канал 1,10.01.2017,22,2,Розничная торговля
3,299,Канал 1,11.01.2017,35,10,Розничная торговля
4,299,Канал 1,12.01.2017,55,4,Розничная торговля
...,...,...,...,...,...,...
808089,311,Канал 13,25.05.2018,63,2,Оптовая торговля
808090,311,Канал 13,05.06.2018,48,6,Оптовая торговля
808091,311,Канал 13,08.06.2018,68,6,Оптовая торговля
808092,311,Канал 13,20.06.2018,51,8,Оптовая торговля


In [10]:
# заменим направлени продаж для удобства цифрами

data["Направление продаж"].replace({"Розничная торговля": "01",
                      "Оптовая торговля": "02"}, inplace=True)

# разобьем дату на отдельные элементы

data[["day", "month", "year"]] = data["Дата накладной"].str.split(".", expand = True)

data.drop(["day",'Дата накладной'], axis=1, inplace=True)

# создадим новую кодировку позиции - код товара+направление продаж, чтобы сагрегировать потом продажи
# в рамках месяца

data['code_sales'] = data['Код товара'].astype(str) + data['Направление продаж']

# удалим лишнее

data.drop(["Код товара",'Канал','Направление продаж'], axis=1, inplace=True)


# агрегируем по продажам в рамках месяца

data_new = data.groupby(['code_sales','year','month','скидка']).sum()
data_new = data_new.reset_index()
data_new['code_sales'] = data_new['code_sales'].astype(int)
data_new

,code_sales,year,month,скидка,"Продажи, шт"
0,10001,2017,01,0,117
1,10001,2017,01,4,54
2,10001,2017,01,6,35
3,10001,2017,01,11,123
4,10001,2017,01,12,28
...,...,...,...,...,...
313863,9902,2018,06,1,94
313864,9902,2018,06,2,113
313865,9902,2018,06,3,95
313866,9902,2018,06,5,53


In [11]:
# попробуем в лоб применить линейную регрессию к датасету, бьем на признаки и целевую переменную
y = data_new['Продажи, шт']
X = data_new.drop(['Продажи, шт'], axis=1)

# также выделим часть данных для проверки качества модели 
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3,shuffle=False)

model_all = LinearRegression()
model_all.fit(X_train, y_train)

LinearRegression()

EDA - посмотреть данные в динамике, построить графики, посмотреть как меняется

In [12]:
data_new['code_sales'].nunique()

1292

In [13]:
X_holdout

,code_sales,year,month,скидка
219707,5102,2018,01,4
219708,5102,2018,01,5
219709,5102,2018,01,6
219710,5102,2018,01,7
219711,5102,2018,01,8
...,...,...,...,...
313863,9902,2018,06,1
313864,9902,2018,06,2
313865,9902,2018,06,3
313866,9902,2018,06,5


In [14]:
print ('r2_score:', r2_score(y_holdout, model_all.predict(X_holdout)))
print ('mean_squared_error:',mean_squared_error(y_holdout, model_all.predict(X_holdout)))
print ('mean_absolute_percentage_error:',mean_absolute_percentage_error(y_holdout, model_all.predict(X_holdout)))

r2_score: 0.010866508511682804
mean_squared_error: 7113.125242548108
mean_absolute_percentage_error: 0.8920089848986149


Видим по метрикам что получившаяся лин модель вообще не описывает зависимости:
   * R2 метрика - лучший результат может быть 1, в нашем случае 0.012, это значит что модель не предсказывает вне зависимости от введенных данных
   * mean_squared_error - среднеквадратичная ошибка, здесь легко понять насколько она большая сравнив значение продаж за месяц одного товара - порядок десятки-сотня и данное значение 7117.54
   * mean_absolute_percentage_error - чем ниже тем лучше, у нас оно имеет очень большое значение, близкое к 1 (применять по идее эту метрику можем, потому что нулевых продаж нет за месяц. Если бы были, то эта метрика не подходит)
  
**Тогда может, попробовать построить лин регрессиую по одному наименованию, и тогда получим сносный результат?**

In [15]:
# возьмем из датасета данныe XY по 1 наименованию 
XY = data_new[data_new['code_sales'] == 10001]

y = XY['Продажи, шт']
X = XY.drop(['Продажи, шт'], axis=1)

# также выделим часть данных для проверки качества модели 
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3,shuffle=False)

model_one = LinearRegression()
model_one.fit(X_train, y_train)

LinearRegression()

In [16]:
print ('r2_score:', r2_score(y_holdout, model_one.predict(X_holdout)))
print ('mean_squared_error:',mean_squared_error(y_holdout, model_one.predict(X_holdout)))
print ('mean_absolute_percentage_error:',mean_absolute_percentage_error(y_holdout, model_one.predict(X_holdout)))

r2_score: -0.7708838198720833
mean_squared_error: 3142.0736275871004
mean_absolute_percentage_error: 0.46970809220521376


Видим, что результаты немного лучше, но все равно у нас так и не получилось построить модель

In [17]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
# print(smf.ols("'Продажи, шт' ~ 'скидка'", data = XY).fit().summary())
print(smf.ols("XY['Продажи, шт'] ~ XY['скидка']", data = XY).fit().summary())

                            OLS Regression Results                            
Dep. Variable:      XY['Продажи, шт']   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.1504
Date:                Mon, 21 Mar 2022   Prob (F-statistic):              0.699
Time:                        20:50:47   Log-Likelihood:                -1092.5
No. Observations:                 212   AIC:                             2189.
Df Residuals:                     210   BIC:                             2196.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       74.2107      5.510     13.469   

На всякий случай еще запустим анализ парной регрессии из статистического пакета и посмотрим:
   * самый главный показатель  R-squared - около нуля (это то количество данных которая наша модель объясняет, ничего не объясняет, значит наша модель не работает)
   * ну и помимо прочего коэф при скидке  -0.1943 - гораздо меньше стандартной ошибки,=>, коэффициент не значим

In [20]:
code = 10001

def one_code_model_qual(code,data):
    models_quality = pd.DataFrame( columns = ['Name', 'r2_score','mean_squared_error',
                                              'mean_absolute_percentage_error'])


    # ohe
    data = pd.get_dummies(data_new, columns=['month','year'], drop_first=True)


    # возьмем из датасета данныe XY по 1 наименованию 
    XY = data[data['code_sales'] == code]

    y = XY['Продажи, шт']
    X = XY.drop(['Продажи, шт'], axis=1)

    # также выделим часть данных для проверки качества модели 
    
    X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3,shuffle=False)

    model_one = LinearRegression()
    model_one.fit(X_train, y_train)

    print(X_train)

    models_quality = models_quality.append({'Name': int(code),
                                            'r2_score': r2_score(y_holdout, model_one.predict(X_holdout)),
                                            'mean_squared_error': mean_squared_error(y_holdout, model_one.predict(X_holdout)),
                                            'mean_absolute_percentage_error': mean_absolute_percentage_error(y_holdout, model_one.predict(X_holdout))}, ignore_index=True)
    return models_quality

In [22]:
one_code_model_qual(code,data=data_new)

     code_sales  скидка  month_02  month_03  month_04  month_05  month_06  \
0         10001       0         0         0         0         0         0   
1         10001       4         0         0         0         0         0   
2         10001       6         0         0         0         0         0   
3         10001      11         0         0         0         0         0   
4         10001      12         0         0         0         0         0   
..          ...     ...       ...       ...       ...       ...       ...   
143       10001      18         0         0         0         0         0   
144       10001      19         0         0         0         0         0   
145       10001       1         0         0         0         0         0   
146       10001       2         0         0         0         0         0   
147       10001       3         0         0         0         0         0   

     month_07  month_08  month_09  month_10  month_11  month_12  year_2018 

,Name,r2_score,mean_squared_error,mean_absolute_percentage_error
0,10001.0,-0.971075,3497.271372,0.498739


In [24]:
from tqdm.notebook import tqdm

In [25]:
codes_from_df = set(data_new['code_sales'])
# results = list(map(lambda x: one_code_model(x,data=data_new),codes_from_df))

models_quality = []

for code in tqdm(codes_from_df):
    try:
        
        models_quality.append(one_code_model(code,data=data_new))
    except:
        print(code)
        continue

8201
8202
16401
16402
24601
24602
32801
32802
41001
41002
49201
49202
57401
57402
65601
65602
101
102
16501
16502
24701
24702
32901
32902
41101
41102
49301
49302
57501
57502
65701
65702
201
202
8401
8402
16601
16602
24801
24802
33001
33002
49401
49402
57601
57602
301
302
8501
8502
16701
16702
24901
24902
33101
33102
41301
41302
49501
49502
57701
57702
401
402
8601
8602
16801
16802
25001
25002
33201
33202
41401
49601
49602
57801
57802
501
502
8701
8702
16901
16902
25101
25102
33301
33302
41501
41502
49701
49702
57901
57902
601
602
8801
8802
17001
17002
25201
25202
33401
33402
41601
41602
49801
49802
58001
58002
701
702
8901
8902
17101
17102
25301
25302
33501
33502
41701
41702
49901
49902
58101
58102
801
802
9001
9002
17201
17202
25401
25402
33601
33602
41801
41802
50001
50002
58201
58202
901
902
9101
9102
17301
17302
25501
25502
33701
33702
41901
41902
50101
50102
58301
58302
1001
1002
9201
9202
17401
17402
25601
25602
33801
33802
42001
42002
50201
50202
58401
58402
1101
1102
9301
9302


In [26]:
models_quality_df = pd.concat(models_quality)
models_quality_df[models_quality_df['r2_score']>0.4]

ValueError: No objects to concatenate

In [27]:
data_forecast = pd.read_excel('C:/Users/Peter/Documents/GitHub/PeterOstr/test_run/Forecast_of_discounts.xlsx')

data_forecast[['del','code']] = data_forecast['Код товара'].str.split("_", expand = True)

# заменим направлени продаж для удобства цифрами

data_forecast["Направление продаж"].replace({"Розничная торговля": "01",
                      "Оптовая торговля": "02"}, inplace=True)

data_forecast.rename(columns={'Month': "month",
                                       'Year': "year"}, inplace=True)


# data_forecast
data_forecast['code_sales'] = data_forecast['code'].astype(str) + data_forecast['Направление продаж']

data_forecast['code_sales'] = data_forecast['code_sales'].astype(int)

data_forecast.drop(["Код товара",'Направление продаж','del','code'], axis=1, inplace=True)

# добавляем коды с нулевой скидкой
code_no_sale = set(data_new['code_sales']) - set(data_forecast['code_sales'])
code_no_sale = list(code_no_sale)
for i in range (len(code_no_sale)):
    new_row = {'code_sales':code_no_sale[i],
               'Скидка':0,
               'month':7,
               'year':2018 }
    data_forecast = data_forecast.append(new_row, ignore_index=True)

data_forecast

,month,year,Скидка,code_sales
0,7,2018,10.000000,101
1,7,2018,7.400000,202
2,8,2018,15.500000,401
3,9,2018,10.857143,502
4,7,2018,12.500000,602
...,...,...,...,...
1563,7,2018,0.000000,22501
1564,7,2018,0.000000,26601
1565,7,2018,0.000000,26602
1566,7,2018,0.000000,30702


In [28]:
new_df = pd.DataFrame(index=pd.MultiIndex.from_tuples(list(product(data_forecast['code_sales'].unique(),data_forecast['month'].unique())),names=['code_sales','month']))
new_df = new_df.join(data_forecast.set_index(['code_sales','month']),how='left')
new_df['year'] = new_df['year'].fillna(2018)
new_df['Скидка'] = new_df['Скидка'].fillna(0)
new_df['year'], new_df['Скидка'] = new_df['year'].astype(int), new_df['Скидка'].astype(int)
new_df.reset_index(inplace=True)

# ohe
new_df = pd.get_dummies(new_df, columns=['month','year'], drop_first=False)
new_df['month_02'], new_df['month_03'], new_df['month_04'], new_df['month_05'], new_df['month_06'], new_df['month_10'], new_df['month_11'], new_df['month_12'] = 0, 0, 0, 0, 0, 0, 0, 0

new_df.rename(columns={'month_7': "month_07",
                              'month_8': "month_08",
                              'month_9': "month_09",
                       'Скидка': "скидка"}, inplace=True)

new_df


,code_sales,скидка,month_07,month_08,month_09,year_2018,month_02,month_03,month_04,month_05,month_06,month_10,month_11,month_12
0,101,10,1,0,0,1,0,0,0,0,0,0,0,0
1,101,0,0,1,0,1,0,0,0,0,0,0,0,0
2,101,0,0,0,1,1,0,0,0,0,0,0,0,0
3,202,7,1,0,0,1,0,0,0,0,0,0,0,0
4,202,0,0,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3871,30702,0,0,1,0,1,0,0,0,0,0,0,0,0
3872,30702,0,0,0,1,1,0,0,0,0,0,0,0,0
3873,34801,0,1,0,0,1,0,0,0,0,0,0,0,0
3874,34801,0,0,1,0,1,0,0,0,0,0,0,0,0


In [29]:
new_df.columns

Index(['code_sales', 'скидка', 'month_07', 'month_08', 'month_09', 'year_2018',
       'month_02', 'month_03', 'month_04', 'month_05', 'month_06', 'month_10',
       'month_11', 'month_12'],
      dtype='object')

In [30]:
data_new.columns

Index(['code_sales', 'year', 'month', 'скидка', 'Продажи, шт'], dtype='object')

In [98]:
prediction = new_df.copy()
predict_value = []
# prediction['predict_value'] = '0'
for i in range(len(prediction)):
    # возьмем из датасета данныe XY по 1 наименованию
    XY = data_new[data_new['code_sales'] == prediction['code_sales'][i]]

    # ohe
    XY = pd.get_dummies(XY, columns=['month','year'], drop_first=True)

    y = XY['Продажи, шт']
    X = XY.drop(['Продажи, шт'], axis=1)
    if set(X.columns) != set(prediction.iloc[[i]].columns):
        difference_btwn_col = set(prediction.iloc[[i]].columns) - set(X.columns)
        for element in difference_btwn_col:
            X[element] = 0

    model_one_line = LinearRegression()
    model_one_line.fit(X, y)
    predict_value.append(int(model_one_line.predict(prediction.iloc[[i]])))
# prediction

E:\anaconda\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
E:\anaconda\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
E:\anaconda\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
E:\anaconda\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an 

  * Нужно построить прогноз продаж на июль-сентябрь 18 с помощью простейшего алгоритма – линейной регрессии.
  * Прогноз нужен на уровне месяца, продукта и направления продаж, соответственно, все данные нужно сначала преобразовать на данный уровень с помощью Pandas.

In [101]:
# prediction
prediction = prediction.drop(['month_02',
                'month_03',
                'month_04',
                'month_05',
                'month_06',
                'month_10',
                'month_11',
                'month_12',
                'year_2018'],axis=1)

In [102]:
prediction['predict_value'] = predict_value
prediction

,code_sales,скидка,month_07,month_08,month_09,predict_value
0,101,10,1,0,0,76
1,101,0,0,1,0,109
2,101,0,0,0,1,94
3,202,7,1,0,0,52
4,202,0,0,1,0,33
...,...,...,...,...,...,...
3871,30702,0,0,1,0,60
3872,30702,0,0,0,1,60
3873,34801,0,1,0,0,119
3874,34801,0,0,1,0,132


In [ ]:
prediction['code'] = prediction['code_sales'][:]

In [123]:
prediction['code_sales'].iloc[::2]

0         101
2         101
4         202
6         401
8         401
        ...  
3866    26601
3868    26602
3870    30702
3872    30702
3874    34801
Name: code_sales, Length: 1938, dtype: int64

In [ ]:
data_forecast["Направление продаж"].replace({"Розничная торговля": "01",
                                             "Оптовая торговля": "02"}, inplace=True)